### Import Required Libraries

In [0]:
import requests
import pandas as pd
import re
import os
import io
from datetime import datetime
from pyspark.sql.types import *

### Generate Dynamic Column List

In [0]:
nums = [str(i) for i in range(0, 150)]
columns = ",".join(nums)

### Read Execution Date from Databricks Widget

In [0]:
dbutils.widgets.text("ODATE", "")
now_str = dbutils.widgets.get("ODATE")
now = datetime.strptime(now_str, "%Y%m%d")

today = now.strftime("%Y%m%d")
year = now.strftime("%Y")
month = now.strftime("%m")
day = now.strftime("%d")

### Build Finviz Request URL and Download Data

In [0]:
filters = 'ind_stocksonly'
token = os.getenv("FINVIZ_TOKEN")
cols = columns

url = f"https://elite.finviz.com/export.ashx?v=151&c={cols}&f={filters}&auth={token}"

response = requests.get(url)
df = pd.read_csv(io.BytesIO(response.content))
df['load_date'] = today

### Save Raw File to Parquet

In [0]:
folder_path = f"/Volumes/raw/finviz/screener_data/{year}/{month}/{day}"
os.makedirs(folder_path, exist_ok=True)

df.to_parquet(fr"{folder_path}/finviz_screener_data_{today}.parquet", engine="pyarrow")

### Define Schema and Convert to Spark

In [0]:
schema = StructType([
    StructField("no", LongType(), True),
    StructField("ticker", StringType(), True),
    StructField("company", StringType(), True),
    StructField("sector", StringType(), True),
    StructField("industry", StringType(), True),
    StructField("country", StringType(), True),
    StructField("market_cap", DoubleType(), True),
    StructField("p_e", DoubleType(), True),
    StructField("forward_p_e", DoubleType(), True),
    StructField("peg", DoubleType(), True),
    StructField("p_s", DoubleType(), True),
    StructField("p_b", DoubleType(), True),
    StructField("p_cash", DoubleType(), True),
    StructField("p_free_cash_flow", DoubleType(), True),
    StructField("dividend_yield", StringType(), True),
    StructField("payout_ratio", StringType(), True),
    StructField("eps_ttm", DoubleType(), True),
    StructField("eps_growth_this_year", StringType(), True),
    StructField("eps_growth_next_year", StringType(), True),
    StructField("eps_growth_past_5_years", StringType(), True),
    StructField("eps_growth_next_5_years", StringType(), True),
    StructField("sales_growth_past_5_years", StringType(), True),
    StructField("eps_growth_quarter_over_quarter", StringType(), True),
    StructField("sales_growth_quarter_over_quarter", StringType(), True),
    StructField("shares_outstanding", DoubleType(), True),
    StructField("shares_float", DoubleType(), True),
    StructField("insider_ownership", StringType(), True),
    StructField("insider_transactions", StringType(), True),
    StructField("institutional_ownership", StringType(), True),
    StructField("institutional_transactions", StringType(), True),
    StructField("short_float", StringType(), True),
    StructField("short_ratio", DoubleType(), True),
    StructField("return_on_assets", StringType(), True),
    StructField("return_on_equity", StringType(), True),
    StructField("return_on_invested_capital", StringType(), True),
    StructField("current_ratio", DoubleType(), True),
    StructField("quick_ratio", DoubleType(), True),
    StructField("lt_debt_equity", DoubleType(), True),
    StructField("total_debt_equity", DoubleType(), True),
    StructField("gross_margin", StringType(), True),
    StructField("operating_margin", StringType(), True),
    StructField("profit_margin", StringType(), True),
    StructField("performance_week", StringType(), True),
    StructField("performance_month", StringType(), True),
    StructField("performance_quarter", StringType(), True),
    StructField("performance_half_year", StringType(), True),
    StructField("performance_year", StringType(), True),
    StructField("performance_ytd", StringType(), True),
    StructField("beta", DoubleType(), True),
    StructField("average_true_range", DoubleType(), True),
    StructField("volatility_week", StringType(), True),
    StructField("volatility_month", StringType(), True),
    StructField("20_day_simple_moving_average", StringType(), True),
    StructField("50_day_simple_moving_average", StringType(), True),
    StructField("200_day_simple_moving_average", StringType(), True),
    StructField("50_day_high", StringType(), True),
    StructField("50_day_low", StringType(), True),
    StructField("52_week_high", StringType(), True),
    StructField("52_week_low", StringType(), True),
    StructField("relative_strength_index_14", DoubleType(), True),
    StructField("change_from_open", StringType(), True),
    StructField("gap", StringType(), True),
    StructField("analyst_recom", DoubleType(), True),
    StructField("average_volume", DoubleType(), True),
    StructField("relative_volume", DoubleType(), True),
    StructField("price", DoubleType(), True),
    StructField("change", StringType(), True),
    StructField("earnings_date", StringType(), True),
    StructField("target_price", DoubleType(), True),
    StructField("ipo_date", StringType(), True),
    StructField("after_hours_close", DoubleType(), True),
    StructField("after_hours_change", StringType(), True),
    StructField("book_sh", DoubleType(), True),
    StructField("cash_sh", DoubleType(), True),
    StructField("dividend", DoubleType(), True),
    StructField("employees", DoubleType(), True),
    StructField("eps_next_q", DoubleType(), True),
    StructField("income", DoubleType(), True),
    StructField("index", StringType(), True),
    StructField("optionable", StringType(), True),
    StructField("prev_close", DoubleType(), True),
    StructField("sales", DoubleType(), True),
    StructField("shortable", StringType(), True),
    StructField("short_interest", DoubleType(), True),
    StructField("float_pct", StringType(), True),
    StructField("open", DoubleType(), True),
    StructField("high", DoubleType(), True),
    StructField("low", DoubleType(), True),
    StructField("trades", LongType(), True),
    StructField("performance_1_minute", StringType(), True),
    StructField("performance_2_minutes", StringType(), True),
    StructField("performance_3_minutes", StringType(), True),
    StructField("performance_5_minutes", StringType(), True),
    StructField("performance_10_minutes", StringType(), True),
    StructField("performance_15_minutes", StringType(), True),
    StructField("performance_30_minutes", StringType(), True),
    StructField("performance_1_hour", StringType(), True),
    StructField("performance_2_hours", StringType(), True),
    StructField("performance_4_hours", StringType(), True),
    StructField("asset_type", DoubleType(), True),
    StructField("etf_type", DoubleType(), True),
    StructField("region", DoubleType(), True),
    StructField("single_category", DoubleType(), True),
    StructField("sector_theme", DoubleType(), True),
    StructField("tags", DoubleType(), True),
    StructField("active_passive", DoubleType(), True),
    StructField("net_expense_ratio", DoubleType(), True),
    StructField("total_holdings", DoubleType(), True),
    StructField("assets_under_management", DoubleType(), True),
    StructField("net_asset_value", DoubleType(), True),
    StructField("net_asset_value_pct", DoubleType(), True),
    StructField("net_flows_1_month", DoubleType(), True),
    StructField("net_flows_pct_1_month", DoubleType(), True),
    StructField("net_flows_3_month", DoubleType(), True),
    StructField("net_flows_pct_3_month", DoubleType(), True),
    StructField("net_flows_ytd", DoubleType(), True),
    StructField("net_flows_pct_ytd", DoubleType(), True),
    StructField("net_flows_1_year", DoubleType(), True),
    StructField("net_flows_pct_1_year", DoubleType(), True),
    StructField("return_1_year", DoubleType(), True),
    StructField("return_3_year", DoubleType(), True),
    StructField("return_5_year", DoubleType(), True),
    StructField("return_10_year", DoubleType(), True),
    StructField("return_since_inception", DoubleType(), True),
    StructField("all_time_high", StringType(), True),
    StructField("all_time_low", StringType(), True),
    StructField("eps_surprise", StringType(), True),
    StructField("revenue_surprise", StringType(), True),
    StructField("exchange", StringType(), True),
    StructField("dividend_ttm", DoubleType(), True),
    StructField("dividend_ex_date", StringType(), True),
    StructField("eps_year_over_year_ttm", StringType(), True),
    StructField("sales_year_over_year_ttm", StringType(), True),
    StructField("52_week_range", StringType(), True),
    StructField("news_time", StringType(), True),
    StructField("news_url", StringType(), True),
    StructField("news_title", StringType(), True),
    StructField("performance_3_years", StringType(), True),
    StructField("performance_5_years", StringType(), True),
    StructField("performance_10_years", StringType(), True),
    StructField("after_hours_volume", DoubleType(), True),
    StructField("eps_growth_past_3_years", StringType(), True),
    StructField("sales_growth_past_3_years", StringType(), True),
    StructField("enterprise_value", DoubleType(), True),
    StructField("ev_ebitda", DoubleType(), True),
    StructField("ev_sales", DoubleType(), True),
    StructField("dividend_growth_1_year", StringType(), True),
    StructField("dividend_growth_3_years", StringType(), True),
    StructField("dividend_growth_5_years", StringType(), True),
    StructField("volume", DoubleType(), True),
    StructField("load_date", StringType(), True)
])

columns = [re.sub(r"\W+", "_", col.replace("%", "pct")).strip("_").lower() for col in df.columns]
df.columns = columns

spark_df = spark.createDataFrame(df, schema=schema)

### Write Data to Delta Table

In [0]:
table_name = "raw.finviz.screener_data"
spark_df.write.format("delta").mode("append").partitionBy("load_date").saveAsTable(table_name)